In [ ]:
import numpy as np
import hyperspy.api as hs
import cv2

def dm3_to_8bit_nparray(file_path):
    # load the dm3 file and returns an 8bit np.array
    file_path = str(file_path)
    data = hs.load(file_path)

    data_arr = np.array(data.data)
    if data_arr.shape [0] < 1300 or data_arr.shape [1] < 1300:
        data_arr = data_arr.astype(np.uint16)
    data_8bit = (data_arr/data_arr.max())*255
    data_8bit = np.uint8(data_8bit)

    return data_8bit

def opposite_angle(angle):
    #return the opposite angle of the variable angle in range [0:2pi], both input and output in radians

    op_angle = angle+np.pi
    return np.mod(op_angle, 2*np.pi)

def get_centered_crop_img(image, center):
    # crops the image to biggest area possible so its get centered on center
    c_x = round(center[0])
    c_y = round(center[1])

    x_lenth = image.shape[0]
    y_lenth = image.shape[1]

    radius = min(c_x, c_y, abs(x_lenth-c_x), abs(y_lenth-c_y))-1

    return image[c_y-radius:c_y+radius, c_x-radius:c_x+radius]

def get_polar_image(img, rad_percent='Nan'):
    # transformates rectangular coordinates to polar, img needs to be centered in center
    if rad_percent == 'Nan':
        rad_percent = (0.06, 0.7)
    elif rad_percent == 'Full':
        rad_percent = (0,1)
    value = np.sqrt(((img.shape[0]/2.0)**2.0)+((img.shape[1]/2.0)**2.0))

    polar_image = cv2.linearPolar(img,(img.shape[0]/2, img.shape[1]/2), value, cv2.WARP_FILL_OUTLIERS)
    polar_image = polar_image.astype(np.uint8)

    return polar_image[:,round(rad_percent[0]*polar_image.shape[1]):round(rad_percent[1]*polar_image.shape[1])]

def axial_sections(pol_img, direc_angle, semi_range, biaxial=True):
    '''
    returns the values of a polar image at the interval
    [direc_angle-semi_range, direc_angle+semi_range] and [direc_angle+np.pi-semi_range, direc_angle+np.pi+semi_range]
    '''    
    l = pol_img.shape[0]
    theta = np.arange(0, 2*np.pi, 2*np.pi/l)

    direc_midle_ind = np.argmin(np.abs(theta-direc_angle))
    cntr_midle_ind = int(np.mod(direc_midle_ind+l/2, l))

    mns_dirct_bound = np.mod(direc_midle_ind-semi_range, l)
    pls_dirct_bound = np.mod(direc_midle_ind+semi_range, l)

    mns_cntr_bound = np.mod(cntr_midle_ind-semi_range, l)
    pls_cntr_bound = np.mod(cntr_midle_ind+semi_range, l)

    if mns_dirct_bound > pls_dirct_bound:
        dirct_section = np.append(pol_img[mns_dirct_bound:], pol_img[:pls_dirct_bound], axis=0)
        cntr_section = pol_img[mns_cntr_bound:pls_cntr_bound]

    elif mns_cntr_bound > pls_cntr_bound:
        dirct_section = pol_img[mns_dirct_bound:pls_dirct_bound]
        cntr_section = np.append(pol_img[mns_cntr_bound:], pol_img[:pls_cntr_bound], axis=0)

    else:
        dirct_section = pol_img[mns_dirct_bound:pls_dirct_bound]
        cntr_section = pol_img[mns_cntr_bound:pls_cntr_bound]
    if biaxial:
        return (dirct_section, cntr_section)
    else:
        return dirct_section

def get_rad_pers(angles, pol_img, semi_range='Full', biaxial=True):
    #semi_range \in [1, Full=90]
    rad_pers = []
    if semi_range == 'Full':
        l = pol_img.shape[0]
        theta = np.arange(0, 2*np.pi, 2*np.pi/l)
        semi_range = round(theta.shape[0]/(round(angles.shape[0]*4)))

    for angle in angles:
        rad_direct = np.sum(axial_sections(pol_img, angle, semi_range)[0], axis=0)/(2*semi_range)
        rad_cntr = np.sum(axial_sections(pol_img, angle, semi_range)[1], axis=0)/(2*semi_range)

        if biaxial:
            rad_pers.append((rad_direct, rad_cntr))
        else:
            rad_pers.append(rad_direct)
    return np.array(rad_pers)

def get_integral_diff(rad_pers, rad_range='Nan'):
    # returns the normalized sqrd value of the area between the curves rad_per[0] and rad_per[1]
    integrals = []
    
    for rad_per in rad_pers:
        rad_direct = rad_per[0]
        rad_cntr = rad_per[1]

        rad_diff = (rad_direct-rad_cntr)**2
        if rad_range == 'Nan':
            integral = np.trapz(rad_diff[round(rad_diff.shape[0]*0.2):round(rad_diff.shape[0]*0.6)])
            integrals.append(integral/rad_diff[round(rad_diff.shape[0]*0.15):round(rad_diff.shape[0]*0.6)].shape[0])
        elif rad_range == 'Full':
            integral = np.trapz(rad_diff)
            integrals.append(integral/rad_diff.shape[0])
        else:
            integral = np.trapz(rad_diff[round(rad_diff.shape[0]*rad_range[0]):round(rad_diff.shape[0]*rad_range[1])])
            integrals.append(integral/rad_diff[round(rad_diff.shape[0]*rad_range[0]):round(rad_diff.shape[0]*rad_range[1])].shape[0])
    
    return np.array(integrals)

In [ ]:
import pandas as pd
import numpy as np
import hyperspy.api as hs
import os
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
import random

# Define a function to oversample the dataset
def oversample_data(images, coordinates, filenames, desired_size):
    current_size = len(images)
    if current_size >= desired_size:
        return images, coordinates, filenames

    # Calculate how many more samples are needed
    additional_samples_needed = desired_size - current_size

    # Randomly duplicate samples
    additional_images = []
    additional_coordinates = []
    additional_filenames = []
    for _ in range(additional_samples_needed):
        index = random.randint(0, current_size - 1)
        additional_images.append(images[index])
        additional_coordinates.append(coordinates[index])
        additional_filenames.append(filenames[index])

    # Append the additional samples
    oversampled_images = np.concatenate((images, additional_images), axis=0)
    oversampled_coordinates = np.concatenate((coordinates, additional_coordinates), axis=0)
    oversampled_filenames = np.concatenate((filenames, additional_filenames), axis=0)
    
    return oversampled_images, oversampled_coordinates, oversampled_filenames

image_dir = 'imagens centro'

# Load images and coordinates
images = []
coordinates = []
filenames = []

for index, row in df.iterrows():
    img_path = os.path.join(image_dir, row['Arquivo'])
    if not os.path.exists(img_path):
        print(f"File {img_path} does not exist")
        continue
    try:
        # Load the dm3 file using hyperspy
        dm3_data = hs.load(img_path)
        img = dm3_data.data
        
        # Check if the image is grayscale and convert to RGB if necessary
        if len(img.shape) == 2:
            img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
        
        # Resize the image
        img = cv2.resize(img, (128, 128))
        
        # Append the processed image, coordinates, and filename
        images.append(img_to_array(img))
        coordinates.append([row['x'], row['y']])
        filenames.append(row['Arquivo'])
    except Exception as e:
        print(f"Could not read {img_path}: {e}")
        continue

if not images:
    raise ValueError("No images were loaded. Check the image directory and file paths.")

# Convert lists to numpy arrays
images = np.array(images, dtype="float") / 255.0
coordinates = np.array(coordinates, dtype="float")
filenames = np.array(filenames)

# Oversample the data
desired_size = 10000  # Set desired size of the dataset
oversampled_images, oversampled_coordinates, oversampled_filenames = oversample_data(images, coordinates, filenames, desired_size)

# Split data into training and testing sets
(trainX, testX, trainY, testY, trainFilenames, testFilenames) = train_test_split(
    oversampled_images, oversampled_coordinates, oversampled_filenames, test_size=0.2, random_state=42)

# Data augmentation
aug = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    horizontal_flip=True,
    fill_mode="nearest")

# Load the pre-trained ResNet50 model + higher level layers
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

# Freeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = False

# Add custom layers on top of the base model
top_model = base_model.output
top_model = GlobalAveragePooling2D()(top_model)
top_model = Dense(128, activation='relu')(top_model)
top_model = Dropout(0.1)(top_model)
top_model = Dense(32, activation='relu')(top_model)
top_model = Dense(2, activation='linear')(top_model)  # Output layer with 2 units (x, y coordinates)

model = Model(inputs=base_model.input, outputs=top_model)
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error', metrics=['mae'])
model.summary()

# Callbacks for early stopping and learning rate reduction
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001)

# Train the model
history = model.fit(aug.flow(trainX, trainY, batch_size=32),
                    validation_data=(testX, testY),
                    epochs=50,
                    callbacks=[early_stopping, reduce_lr])

# Evaluate the model
loss, mae = model.evaluate(testX, testY)
print(f"Test Loss: {loss}, Test MAE: {mae}")

# Make predictions
predictions = model.predict(testX)
int_predictions = np.round(predictions).astype(int)  # Round and convert to integer

# Display the first 5 predictions and their corresponding filenames
for i in range(5):  # Display the first 5 predictions
    print(f"File: {testFilenames[i]}")
    print(f"Predicted: {2 * int_predictions[i]}, Actual: {testY[i]}")

In [ ]:
# Make predictions
predictions = model.predict(testX)
int_predictions = np.round(predictions).astype(int)  # Round and convert to integer

# Display the first 5 predictions and their corresponding filenames
for i in range(5):  # Display the first 5 predictions
    print(f"File: {testFilenames[i]}")
    print(f"Predicted: {int_predictions[i]}, Actual: {testY[i]}")

In [ ]:
import hyperspy.api as hs
import matplotlib.pyplot as plt

# Load the image
a = hs.load('imagens centro/285 mm 64f 1s  Fe3O4.dm3')
image_data = a.data

# Make predictions
predictions = model.predict(testX)
int_predictions = np.round(predictions).astype(int)  # Round and convert to integer

# Display the first 5 predictions and their corresponding filenames
for i in range(5):
    # Plot the image
    plt.imshow(image_data, cmap='gray')

    # Add a marker at the specified coordinates (adjust coordinates as needed)
    plt.scatter(int_predictions[i][0], int_predictions[i][1], color='red', marker='o', s=10)
    plt.scatter(testY[i][0], testY[i][1], color='blue', marker='o', s=10)

    # Show the plot
    plt.title(f"File: {testFilenames[i]} | Predicted: {int_predictions[i]}, Actual: {testY[i]}")
    plt.show()